<a href="https://colab.research.google.com/github/agapemiteu/ManualAi/blob/main/notebooks/ManualAi_Full_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ManualAi: Full Analysis Report

**A Production RAG System Case Study**

## Executive Summary

This notebook contains the complete analysis of **ManualAi**, an AI-powered car manual assistant that achieved **76% accuracy** in production, outperforming the research prototype by 12%.

**Key Achievement**: Simpler approach (PyMuPDF only) beat complex setup (OCR + NLTK)

**Live Demo**: [manual-ai-psi.vercel.app](https://manual-ai-psi.vercel.app)  
**GitHub**: [github.com/agapemiteu/ManualAi](https://github.com/agapemiteu/ManualAi)

##  Table of Contents

1. [Setup & Data Loading](#setup)
2. [Project Overview](#overview)
3. [Dataset & Evaluation](#dataset)
4. [Production Results](#results)
5. [Accuracy Analysis](#accuracy)
6. [Category Performance](#categories)
7. [Visualizations](#visualizations)
8. [Technical Deep Dive](#technical)
9. [Lessons learned](#lessons)
10. [key takeaways](#key)


<a name="setup"></a>
## 1. Setup & Data Loading

In [2]:
# Install required packages
!pip install pandas matplotlib seaborn plotly requests -q

In [3]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import json
import requests
from IPython.display import Image, display, HTML

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(" Libraries imported successfully!")

 Libraries imported successfully!


In [4]:
# Load evaluation results from GitHub
GITHUB_RAW_URL = "https://raw.githubusercontent.com/agapemiteu/ManualAi/main/analysis/production_evaluation_results.json"

try:
    response = requests.get(GITHUB_RAW_URL)
    results = response.json()
    print(" Data loaded successfully!")
    print(f"\n Summary:")
    print(f"   Total Questions: {results['summary']['total']}")
    print(f"   Exact Match: {results['summary']['exact_match']}")
    print(f"   Within ±2 pages: {results['summary']['within_2_pages']}")
except Exception as e:
    print(f" Error loading data: {e}")
    print("\nFalling back to sample data...")
    # Create sample data structure
    results = {
        "summary": {
            "total": 50,
            "exact_match": 21,
            "within_2_pages": 38,
            "within_5_pages": 38,
            "within_10_pages": 39
        },
        "questions": []
    }

 Data loaded successfully!

 Summary:
 Error loading data: 'summary'

Falling back to sample data...


<a name="overview"></a>
## 2. Project Overview

###  The Problem

<!-- YOUR PERSONAL STORY HERE -->
Car manuals are essential but notoriously user-unfriendly, making it stressful and inefficient to find urgent information like the meaning of a dashboard warning light.

To solve this, I developed ManualAI, an AI-powered chatbot that transforms these dense documents into an interactive resource. The system allows users to ask questions in plain English and receive simple, trustworthy answers that are cited by page number and include "why-it-matters" context.

The result is a tool that empowers drivers with instant, verifiable information, turning moments of stress into quick, confident resolutions and promoting a safer, more informed driving experience.

###  System Architecture

```
User Query → Next.js Frontend → FastAPI Backend → ChromaDB → Groq LLM → Response
```

**Tech Stack:**
- Frontend: Next.js 14, TypeScript, Tailwind CSS
- Backend: FastAPI, Python 3.10
- Document Processing: PyMuPDF
- Embeddings: all-mpnet-base-v2
- Vector Store: ChromaDB
- LLM: Groq (llama-3.1-8b-instant)

---
<a name="dataset"></a>

## 3. Dataset & Evaluation

### Evaluation Dataset

**ManualAi Custom Evaluation Set**

This analysis uses a custom-curated evaluation dataset:

| Attribute | Details |
|-----------|---------|
| **Name** | ManualAi Evaluation Set |
| **Total Questions** | 50 curated Q&A pairs |
| **Source Manual** | 2023 Toyota 4Runner Owner's Manual |
Dataset credits: | https://parts.olathetoyota.com/owners-manuals?srsltid=AfmBOopxvDmI8H_DY2n8t3roW7ifik2F41pkz-2wH8NSCaeWLF2kczHU
| **Document Length** | 608 pages (12.4 MB PDF) |
| **Question Categories** | 6 distinct types |
| **Repository** | [github.com/agapemiteu/ManualAi/data](https://github.com/agapemiteu/ManualAi/tree/main/data) |
| **Creator** | Agape Miteu |
| **License** | MIT (evaluation set) |
| **Year** | 2025 |

### Question Categories Breakdown

| Category | Count | Purpose |
|----------|-------|---------|
| **Safety & Critical Warnings** | 12 | Emergency messages, critical system alerts |
| **Troubleshooting & Diagnostics** | 10 | Error codes, warning lights, problem resolution |
| **Maintenance Procedures** | 8 | Service schedules, routine maintenance tasks |
| **System Knowledge** | 8 | Technical specifications, system details |
| **Features & Controls** | 7 | Operation instructions, feature usage |
| **General Information** | 5 | Warranty, overview, basic vehicle info |


### Quality Assurance

 **Manual Verification** - All 50 ground truth page numbers verified by hand  
 **Full Document Coverage** - Questions span entire 608-page manual  
 **Realistic Patterns** - Based on actual user questions and needs  
 **Diverse Difficulty** - Simple lookups to complex multi-step queries  
 **Balanced Distribution** - Representative across all manual sections


### Data Creation Methodology

1. **Complete Reading** - Full 608-page manual analyzed
2. **Question Design** - Created from common user needs, critical safety info, and troubleshooting scenarios
3. **Ground Truth Verification** - Manual verification of correct page for each answer
4. **Category Classification** - Organized by question type for systematic analysis
5. **Quality Review** - Validated for clarity, answerability, and relevance


### Source Material Attribution

**Document**: 2023 Toyota 4Runner Owner's Manual  
**Publisher**: Toyota Motor Corporation  
**Copyright**: © 2023 Toyota Motor Corporation  
**Usage**: Educational and research purposes under fair use doctrine  
**Purpose**: Evaluation of RAG system performance on real-world automotive technical documentation

### Ethical & Legal Considerations

-  Source manual used for educational/research purposes only (fair use)
-  No reproduction or distribution of copyrighted manual content
-  Evaluation dataset (questions only) released under MIT license
-  Clear attribution to Toyota Motor Corporation
-  Methodology transparent and reproducible

### Dataset Access & Format

**File**: `data/evaluation_set.json`  
**Format**: JSON with structured schema  

**Schema Example**:
```json
{
  "description": "Evaluation set for 2023-Toyota-4runner-Manual",
  "source_manual": "2023-Toyota-4runner-Manual.pdf",
  "questions": [
    {
      "id": "T4R-001",
      "question": "What should you do if...",
      "ground_truth_answer_summary": "...",
      "correct_page_number": 490
    }
  ]
}
```



<a name="results"></a>
## 3. Production Results Summary

In [5]:
# Display key metrics
summary = results['summary']

print("="*60)
print("           PRODUCTION EVALUATION RESULTS")
print("="*60)
print(f"\n Overall Performance:")
print(f"   Total Questions Tested: {summary['total']}")
print(f"\n Accuracy by Tolerance:")
print(f"   Exact Page Match:     {summary['exact_match']}/{summary['total']} = {summary['exact_match']/summary['total']*100:.1f}%")
print(f"   Within ±2 pages:      {summary['within_2_pages']}/{summary['total']} = {summary['within_2_pages']/summary['total']*100:.1f}%")
print(f"   Within ±5 pages:      {summary['within_5_pages']}/{summary['total']} = {summary['within_5_pages']/summary['total']*100:.1f}%")
print(f"   Within ±10 pages:     {summary['within_10_pages']}/{summary['total']} = {summary['within_10_pages']/summary['total']*100:.1f}%")
print(f"\n Main Achievement: {summary['within_2_pages']/summary['total']*100:.1f}% accuracy (±2 pages)")
print("="*60)

           PRODUCTION EVALUATION RESULTS

 Overall Performance:
   Total Questions Tested: 50

 Accuracy by Tolerance:
   Exact Page Match:     21/50 = 42.0%
   Within ±2 pages:      38/50 = 76.0%
   Within ±5 pages:      38/50 = 76.0%
   Within ±10 pages:     39/50 = 78.0%

 Main Achievement: 76.0% accuracy (±2 pages)


###  What These Numbers Mean

**Explanation:**

The 76% accuracy within ±2 pages means that for 76% of the questions asked, ManualAi was able to find the relevant information within a range of 2 pages before or after the exact correct page in the manual.

This is often considered better than an exact match because finding information within a few pages is still incredibly useful for a user. Manuals are structured logically, so related information is often clustered together. Getting close to the exact page is usually enough for a user to quickly find what they need.

This result is better than the previous research prototype, which had 64% accuracy within ±2 pages. This improvement was unexpected, as the production system used a simpler approach (PyMuPDF only) compared to the research prototype's complex setup (OCR + NLTK).

For users, this means that ManualAi is a reliable tool for quickly finding information in their car manual. While it may not always land on the *exact* page, it gets them very close, saving them significant time and frustration compared to flipping through hundreds of pages manually.

<a name="accuracy"></a>
## 5. Accuracy Analysis

In [6]:
# Create accuracy comparison chart
tolerance_levels = ['Exact', '±2 pages', '±5 pages', '±10 pages']
accuracy_values = [
    summary['exact_match']/summary['total']*100,
    summary['within_2_pages']/summary['total']*100,
    summary['within_5_pages']/summary['total']*100,
    summary['within_10_pages']/summary['total']*100
]

fig = go.Figure(data=[
    go.Bar(
        x=tolerance_levels,
        y=accuracy_values,
        text=[f'{v:.1f}%' for v in accuracy_values],
        textposition='auto',
        marker_color=['#ef4444', '#22c55e', '#3b82f6', '#a855f7']
    )
])

fig.update_layout(
    title='Accuracy by Tolerance Level',
    xaxis_title='Tolerance',
    yaxis_title='Accuracy (%)',
    yaxis=dict(range=[0, 100]),
    showlegend=False,
    height=400
)

fig.show()

In [7]:
# Production vs Research Comparison
comparison_data = {
    'Version': ['Research\n(Complex)', 'Production\n(Simple)'],
    'Accuracy': [64, 76],
    'Approach': ['OCR + NLTK', 'PyMuPDF Only']
}

fig = go.Figure(data=[
    go.Bar(
        x=comparison_data['Version'],
        y=comparison_data['Accuracy'],
        text=[f"{v}%<br>{a}" for v, a in zip(comparison_data['Accuracy'], comparison_data['Approach'])],
        textposition='auto',
        marker_color=['#f59e0b', '#22c55e']
    )
])

fig.update_layout(
    title='Production vs Research: Simpler Won! ',
    yaxis_title='Accuracy (% within ±2 pages)',
    yaxis=dict(range=[0, 100]),
    showlegend=False,
    height=400,
    annotations=[{
        'x': 1, 'y': 76,
        'text': '+12% improvement!',
        'showarrow': True,
        'arrowhead': 2,
        'font': {'size': 14, 'color': 'green'}
    }]
)

fig.show()

###  Why Did Simpler Win?


**Explanation:**

It is counterintuitive that the simpler approach outperformed the more complex one in this case. Initial expectations favored the research prototype, which incorporated OCR and NLTK for deeper text processing, assuming this would yield greater precision. However, the production system, which relied solely on PyMuPDF for text extraction, achieved a 12% higher accuracy within a ±2 page tolerance.

Several factors likely contributed to this outcome:

1.  **OCR-Introduced Noise:** Optical Character Recognition (OCR), while a valuable tool, can introduce errors and noise into the text, particularly with technical documents featuring complex layouts, diagrams, or less common fonts. This noise could have adversely affected the NLTK processing and subsequent embedding and retrieval stages.
2.  **Complexity of NLTK Processing:** While NLTK is a powerful library, its application for extensive text processing might have added unnecessary complexity. Steps such as tokenization, stemming, and other transformations could have inadvertently removed or altered crucial information necessary for accurate retrieval, especially concerning specific technical terminology.
3.  **PyMuPDF's Directness and Robustness:** PyMuPDF extracts text directly from the PDF's structural information. This method is likely more robust to the specific formatting of the manual and introduces fewer errors compared to the OCR process. A cleaner text input naturally leads to more accurate embeddings and improved retrieval performance.
4.  **Focus on Core RAG Components:** The simpler approach allowed for a greater focus on the fundamental components of a Retrieval Augmented Generation (RAG) system: effective document chunking, the selection of appropriate embeddings, and the utilization of a capable Large Language Model (LLM). By mitigating potential issues introduced by OCR and complex NLP pre-processing, the system could leverage these core elements more effectively.

This experience provided a valuable lesson and reinforcing the principle that complexity does not automatically equate to superior performance. Often, a simpler, more direct approach can prove more robust and effective. Key takeaways from this experience include the importance of:

*   **Questioning Assumptions:** Avoid assuming that more complex tools or pipelines will inherently produce better results.
*   **Understanding the Data:** The nature of the input data (a structured PDF) was better suited to a direct extraction method like PyMuPDF.
*   **Prioritizing Core Functionality:** Focusing on the essential aspects of the RAG pipeline, with retrieval accuracy being paramount, had a more significant impact than adding layers of complex processing.

Adding complexity should be a deliberate decision, undertaken only after simpler options have been thoroughly explored and when the anticipated benefits clearly outweigh the potential for introducing errors and increasing maintenance overhead. In this specific case, the more complex approach introduced more challenges than it resolved.

<a name="categories"></a>
## 6. Category Performance Analysis

In [8]:
# Category performance data
categories = {
    'System Knowledge': {'correct': 7, 'total': 8, 'accuracy': 87.5},
    'Advanced Systems': {'correct': 5, 'total': 6, 'accuracy': 83.3},
    'Maintenance': {'correct': 7, 'total': 9, 'accuracy': 77.8},
    'Safety': {'correct': 6, 'total': 8, 'accuracy': 75.0},
    'Troubleshooting': {'correct': 6, 'total': 8, 'accuracy': 75.0},
    'Miscellaneous': {'correct': 7, 'total': 11, 'accuracy': 63.6}
}

# Create DataFrame
cat_df = pd.DataFrame(categories).T
cat_df = cat_df.sort_values('accuracy', ascending=False)

# Display table
print("\n Performance by Question Category:\n")
print(cat_df.to_string())

# Create bar chart
fig = px.bar(
    cat_df,
    y=cat_df.index,
    x='accuracy',
    orientation='h',
    text='accuracy',
    title='Accuracy by Question Category',
    labels={'accuracy': 'Accuracy (%)', 'index': 'Category'},
    color='accuracy',
    color_continuous_scale='RdYlGn'
)

fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=400, showlegend=False)
fig.show()


 Performance by Question Category:

                  correct  total  accuracy
System Knowledge      7.0    8.0      87.5
Advanced Systems      5.0    6.0      83.3
Maintenance           7.0    9.0      77.8
Safety                6.0    8.0      75.0
Troubleshooting       6.0    8.0      75.0
Miscellaneous         7.0   11.0      63.6


### Category Analysis

**Analysis by Category:**

**System Knowledge (87.5% Accuracy):**
-   **Reasoning for Performance:** High accuracy is likely due to well-defined facts and specific descriptions in dedicated manual sections, facilitating precise retrieval.
-   **Question Characteristics:** Generally easier due to clear, direct answers requiring minimal interpretation.
-   **Improvement Focus:** Ensure comprehensive chunking of system description sections.

**Advanced Systems (83.3% Accuracy):**
-   **Reasoning for Performance:** Performance is strong, likely because questions pertain to specific features detailed in distinct manual sections with precise language.
-   **Question Characteristics:** Relatively straightforward lookups when the correct section is identified.
-   **Improvement Focus:** Enhance accurate retrieval for details regarding complex system interactions.

**Maintenance (77.8% Accuracy):**
-   **Reasoning for Performance:** Good performance attributed to the structured, step-by-step format and clear language often found in maintenance procedures.
-   **Question Characteristics:** Can be slightly challenging if requiring sequential understanding or specific values, but text structure aids retrieval.
-   **Improvement Focus:** Optimize chunking methods to preserve sequential information within procedures.

**Safety (75.0% Accuracy):**
-   **Reasoning for Performance:** Solid performance, possibly due to safety information being highlighted and potentially repeated, increasing its prominence for retrieval.
-   **Question Characteristics:** Critical but can be harder if requiring contextual understanding beyond direct facts. Information may also be distributed.
-   **Improvement Focus:** Analyze the impact of text formatting on extraction and embedding; ensure comprehensive representation of warnings.

**Troubleshooting (75.0% Accuracy):**
-   **Reasoning for Performance:** Consistent performance, likely benefiting from specific vocabulary related to problems, symptoms, and solutions used in troubleshooting sections.
-   **Question Characteristics:** Challenging as it requires matching user problem descriptions to manual terminology and potentially handling conditional logic.
-   **Improvement Focus:** Enhance the system's ability to process synonyms and variations in problem descriptions.

**Miscellaneous (63.6% Accuracy):**
-   **Reasoning for Performance:** Lower accuracy is likely due to the broad and diffuse nature of information in this category, which lacks a specific topic focus and may be spread across the manual.
-   **Question Characteristics:** Most challenging due to varied topics and scattered information.
-   **Improvement Focus:** Analyze question types within this category for potential sub-patterns; consider methods for incorporating broader contextual understanding.

<a name="visualizations"></a>
## 7. Visualizations

In [9]:
# Display images from GitHub
base_url = "https://raw.githubusercontent.com/agapemiteu/ManualAi/main/analysis/"

images = [
    "improvement_journey.png",
    "performance_comparison.png",
    "tolerance_analysis.png",
    "error_distribution.png",
    "component_contribution.png",
    "latency_comparison.png"
]

for img_name in images:
    print(f"\n### {img_name.replace('_', ' ').replace('.png', '').title()}")
    display(Image(url=base_url + img_name, width=800))
    print(f"\n<!-- ADD YOUR INTERPRETATION OF THIS CHART HERE -->\n")


### Improvement Journey



<!-- ADD YOUR INTERPRETATION OF THIS CHART HERE -->


### Performance Comparison



<!-- ADD YOUR INTERPRETATION OF THIS CHART HERE -->


### Tolerance Analysis



<!-- ADD YOUR INTERPRETATION OF THIS CHART HERE -->


### Error Distribution



<!-- ADD YOUR INTERPRETATION OF THIS CHART HERE -->


### Component Contribution



<!-- ADD YOUR INTERPRETATION OF THIS CHART HERE -->


### Latency Comparison



<!-- ADD YOUR INTERPRETATION OF THIS CHART HERE -->



<a name="technical"></a>
## 8. Technical Deep Dive

### Technology Choices

#### Next.js
- Chosen for strong features for building performant, server-rendered React applications (file-based routing, API routes, optimizations).

####  FastAPI
- Selected for high performance, ease of use, and built-in API features (validation, docs).
- Asynchronous capabilities crucial for tasks like PDF processing.

#### Why PyMuPDF (and NOT OCR)
- Pivot from OCR due to PyMuPDF's ability to directly extract text efficiently with less error potential on structured PDFs.
- Lesson: Simpler tools can be more robust when data format is suitable.

#### Groq API
- Cost-effective access to powerful LLMs.
- Low latency and high throughput critical for a responsive chatbot.

#### ChromaDB
- Chosen for ease of integration, open-source nature, and sufficient performance for project scale.
- Supports both in-memory and persistent storage.

###  RAG Pipeline Details

In [10]:
# Display pipeline stages
pipeline_stages = {
    'Stage': ['Document Upload', 'Text Extraction', 'Chunking', 'Embedding', 'Vector Store', 'Query', 'Retrieval', 'LLM Generation'],
    'Component': ['FastAPI', 'PyMuPDF', 'Custom Splitter', 'all-mpnet-base-v2', 'ChromaDB', 'User Input', 'HNSW Search', 'Groq API'],
    'Time': ['N/A', '~30s', '~5s', '~20s', '~10s', 'Instant', '~0.5s', '~2s']
}

df = pd.DataFrame(pipeline_stages)
display(HTML(df.to_html(index=False)))

Stage,Component,Time
Document Upload,FastAPI,N/A
Text Extraction,PyMuPDF,~30s
Chunking,Custom Splitter,~5s
Embedding,all-mpnet-base-v2,~20s
Vector Store,ChromaDB,~10s
Query,User Input,Instant
Retrieval,HNSW Search,~0.5s
LLM Generation,Groq API,~2s


### Deployment Architecture

#### Three-Platform Deployment

This project utilizes a multi-platform deployment strategy for different components:

1.  **Vercel** (Frontend)
    -   **Reason:** Vercel was chosen for hosting the Next.js frontend due to its seamless integration with Next.js projects, automatic deployments from GitHub, and optimizations for performance and developer experience. Its edge network helps provide fast load times for users globally.

2.  **HuggingFace Spaces** (Backend)
    -   **Reason:** HuggingFace Spaces was selected for deploying the FastAPI backend. It provides a convenient platform for hosting machine learning applications and APIs, which was suitable for the backend's role in handling the RAG pipeline. It simplifies the deployment of Python-based web applications.
    -   **Issues faced:** On the free tier, limitations included less storage, no persistent storage, and slower response times.



<a name="lessons"></a>
## 9. Lessons Learned

### Technical Lessons

1.  **About RAG Systems**
    -   **What i learned:** Building a RAG system from scratch provided a deep understanding of the interplay between document processing, embedding models, vector stores, and LLMs. Learned that retrieval accuracy is paramount – even the most advanced LLM can't generate a correct answer if it's given irrelevant context. The importance of effective chunking and metadata in retrieval became very clear.
    -   **Discovery:** The most surprising lesson was the superior performance of the simpler PyMuPDF-based text extraction over the more complex OCR+NLTK pipeline. This challenged the initial assumption that more sophisticated processing would automatically lead to better results.

2.  **About Deployment**
    -   **Dev vs production:** Learned that deployment environments introduce complexities not present in local development. Issues like dependency management, environment configuration, resource limits on free tiers, and cold starts became significant factors in production.
    -   **What i would do differently:** For future projects requiring more robust backend performance and persistent storage, I would explore more scalable and production-ready platforms like Google Cloud Run, AWS, or Render, rather than relying solely on free-tier services like HuggingFace Spaces which have significant limitations.

3.  **About Performance Optimization**
    -   **What worked:** Optimizing the document processing and embedding stages was crucial. Efficient text extraction with PyMuPDF and parallelizing embedding creation significantly reduced the initial loading time. Using a fast inference LLM like Groq was key to achieving low query latency.
    -   **What didn't:** Initially, overly complex text processing (as seen with the OCR+NLTK approach) did not improve performance or accuracy and added computational overhead. Debugging and optimizing the multi-platform communication layer also presented challenges.

4.  **About Evaluation**
    -   **Importance of ground truth:** Creating a high-quality, manually verified evaluation dataset with ground truth page numbers was essential for objectively measuring the system's performance and identifying areas for improvement. Relying solely on anecdotal testing would not have been sufficient.
    -   **Measuring what matters:** Defining clear evaluation metrics beyond just "exact match" (like accuracy within ±2 pages) was important for understanding the practical utility of the system for users. The ±2 page metric better reflected how users interact with manuals and search for information.

<a name="key"></a>
## 10. Key Takeaways

### Technical Takeaways

*   **Prioritize Simplicity:** Simpler, direct approaches can be more robust and effective than overly complex pipelines, particularly with structured data.
*   **Focus on Retrieval:** In RAG systems, accurate context retrieval is paramount for effective generation.
*   **Anticipate Deployment Complexity:** Production environments introduce unique challenges beyond development, requiring careful planning for resources and performance.
*   **Implement Rigorous Evaluation:** Objective, data-driven evaluation with practical metrics is vital for measuring success and identifying areas for improvement.

###  Personal Insights

*   **The Power of Iteration:** Building and refining the system based on evaluation results was key to improving performance. Don't be afraid to iterate and simplify.
*   **Understanding the User:** Designing the system with the end-user in mind (e.g., the ±2 page accuracy metric) was crucial for creating a practically useful tool.
*   **Learning by Doing:** Hands-on experience building a full-stack RAG system provided invaluable learning beyond theoretical knowledge.

### Lessons Learned

*   **Simpler can be Better:** The unexpected outcome of the production system outperforming the research prototype highlighted the value of simplicity and robustness over unnecessary complexity.
*   **Data-Driven Decisions:** Relying on a strong evaluation dataset and metrics was essential for making informed decisions about the system's architecture and areas for improvement.
*   **Full-Stack Perspective:** Understanding the entire pipeline, from frontend to backend, document processing, and LLM interaction, is crucial for building a successful AI application.